In [15]:
import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go

from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.optim import optimize_acqf
print(0)

0


In [16]:
def target_function1d(individuals):
    result = []
    for x in individuals:
        result.append(
            np.exp(-(x[0] - 2)**2) + np.exp(-(x[0] - 6)**2/10) + 1/ (x[0]**2 + 1)
        )
    return torch.tensor(result)

In [17]:
x = np.linspace(-2., 10., 100)
x_new = x.reshape((100, -1))
z = target_function1d(x_new)

data = go.Scatter(x=x, y=z)
fig = go.Figure(data=data)
fig.update_layout(title="Example 1D", xaxis_title="input", yaxis_title="output")
fig.show()

In [18]:
def gen_initial_data(n=10):
    train_x = torch.rand(n, 1) # [n x 1 x 1]
    exact_obj = target_function1d(train_x).unsqueeze(-1) # [n x 1 x 1]
    best_observed_value = exact_obj.max().item()
    return train_x, exact_obj, best_observed_value

In [19]:
def gen_next_points(init_x, init_y, best_init_y, bounds, n_points=1):
    single_model = SingleTaskGP(init_x, init_y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)

    EI = qExpectedImprovement(model=single_model, best_f=best_init_y)

    candidates, _ = optimize_acqf(acq_function=EI,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=200,
                                  raw_samples=512,
                                  options={"batch_limit": 5, "maxiter": 200})
    return candidates

In [20]:
n_runs = 16

init_x, init_y, best_init_y = gen_initial_data(20)
bounds = torch.tensor([[0.], [10.]])

for i in range(n_runs):
    print(f"Nr. of optimization run: {i}")
    new_candidates = gen_next_points(init_x, init_y, best_init_y, bounds, 1)
    new_results = target_function1d(new_candidates).unsqueeze(-1)

    print(f"New candidate is: {new_candidates}")
    init_x = torch.cat([init_x, new_candidates])
    init_y = torch.cat([init_y, new_results])
    best_init_y = init_y.max().item()
    print(f"Best point performs this way: {best_init_y}")

Nr. of optimization run: 0
New candidate is: tensor([[2.7578]])
Best point performs this way: 1.0488195419311523
Nr. of optimization run: 1
New candidate is: tensor([[5.7847]])
Best point performs this way: 1.0488195419311523
Nr. of optimization run: 2
New candidate is: tensor([[8.6592]])
Best point performs this way: 1.0488195419311523
Nr. of optimization run: 3
New candidate is: tensor([[4.3954]])
Best point performs this way: 1.0488195419311523
Nr. of optimization run: 4
New candidate is: tensor([[1.9797]])
Best point performs this way: 1.4015085697174072
Nr. of optimization run: 5
New candidate is: tensor([[6.9660]])
Best point performs this way: 1.4015085697174072
Nr. of optimization run: 6
New candidate is: tensor([[10.]])
Best point performs this way: 1.4015085697174072
Nr. of optimization run: 7
New candidate is: tensor([[2.1025]])
Best point performs this way: 1.4015085697174072
Nr. of optimization run: 8
New candidate is: tensor([[1.9636]])
Best point performs this way: 1.401